##### Metody obliczeniowe w nauce i technice

## Laboratorium 8 - Page Rank

### Sprawozdanie sporządził: Marcin Zielonka

### Wstęp

Do realizacji zadań skorzystam z gotowych funkcjonalności zawartych w bibliotekach:
* `numpy` - wersja `1.18.2`
* `scipy` - wersja `1.4.1`
* `math`
* `random`

In [1]:
import numpy as np
import scipy.linalg
import random
import math

Wybór losowego wierzchołka z uwzględnieniem ich rang przeprowadzę przy użyciu drzewa BST. Na ten cel zaimplementowałem odpowiednie struktury i funkcje:

In [2]:
class Node:
    "tree"
    def __init__(self, key, val, left=None, right=None):
        self.key = key
        self.val = val
        self.left = left
        self.right = right
        
    def __str__(self):
        return str(self.val)

In [3]:
def build_tree(values):
    root = Node(key=values[0][0], val=values[0][1])
    
    for key, val in values[1:]:
        previous = root
        node = root
    
        while node != None:
            previous = node
            
            if val < node.val:
                node = node.left
            else:
                node = node.right
        if val < previous.val:
            previous.left = Node(key=key, val=val)
        else:
            previous.right = Node(key=key, val=val)
            
    return root

In [4]:
def upper_key(tree, val):
    if tree.left is None and tree.right is None and tree.val < val:
        return None
    
    if tree.val >= val and tree.left is None or tree.val >= val and tree.left.val < val:
        return tree.key
    
    if tree.val <= val:
        return upper_key(tree.right, val)
    else:
        return upper_key(tree.left, val)

Dodatkowo zdefiniuję funkcje do manipulacji grafami:

Funkcja tworząca graf skierowany silnie spójny o $n$ wierzchołkach w postaci macierzy sąsiedztwa:

In [5]:
def generate_graph(n):
    A = np.ones(n*n).reshape(n,n)
    
    for i in range(n):
        A[i][i] = 0
        
    return A

Funkcja ładująca graf w postaci macierzy sąsiedztwa z pliku:

In [6]:
def load_graph(filename):
    f = open(filename)
    content = f.readlines()
    
    size = int(content[0])
    A = np.zeros(size * size).reshape(size, size)
    
    for connection in content[1:]:
        u = int(connection.split('\t')[0])
        v = int(connection.split('\t')[1])
        
        A[u][v] = 1
        
    return A

Funkcja skalująca macierz sąsiedztwa $A$ zgodnie z zależnością:

$$
A_{u,v} = \left\{ \begin{array}{ll}
\frac{1}{N_u} & \textrm{jeśli krawędź $(u,v)$ istnieje}\\
0 & \textrm{w przeciwnym wypadku}\\
\end{array} \right.
$$

In [7]:
def transform_graph(A):
    n = len(A)
    B = np.zeros(n*n).reshape(n,n)
    
    for u in range(n):
        for v in range(n):
            if A[u][v] == 0:
                continue
            B[u][v] = 1 /  np.sum(A[v])
    
    return B

Funkcja, która symuluje przechodzenie po grafie $A$ z uwzględnieniem rang wierzchołków:

In [8]:
def graph_inspect(A, ranks, steps):
    n = len(A)
    trace = [0]
    
    for i in range(steps):
        nodes = np.array([(idx, ranks[idx]) for (idx, x) in enumerate(A[trace[-1]]) if x != 0])
                
        for i in range(1, len(nodes)):
            nodes[i][1] += nodes[i - 1][1]
        
        nodes = np.array([(key, val / nodes[-1][1]) for (key, val) in nodes])
        
        tree_rank = build_tree(nodes)
        trace.append(int(upper_key(tree_rank, random.random())))
    
    occurences = [trace.count(node) for node in np.arange(0, n)]
    
    return trace, occurences

### Zadanie 1: Prosty ranking wierzchołków

Zaimplementuj prosty model błądzenia przypadkowego po grafie skierowanym:

$$\mathbf{r}(u)=d\sum_{v\in{B_u}}{\frac{\mathbf{r}(v)}{N_v}}$$

gdzie $\mathbf{r}(u)$ oznacza ranking wierzchołka $u$, parametr $c$ jest używany w normalizacji, $B_u$ jest zbiorem wierzchołków, z których wychodzą krawędzie do wierzchołka $u$, $F_v$ oznacza zbiór wierzchołków, do których dochodzą krawędzie z wierzchołka $v$, a $N_v=|F_v|$. W zapisie macierzowym:

$$\mathbf{r}=d\mathbf{Ar}$$

gdzie $\mathbf{A}$ jest macierzą adiacencji grafu, w której każdy wiersz $u$ jest przeskalowany wyjściowym stopniem wierzchołka $u$.

$$
A_{u,v} = \left\{ \begin{array}{ll}
\frac{1}{N_u} & \textrm{jeśli krawędź $(u,v)$ istnieje}\\
0 & \textrm{w przeciwnym wypadku}\\
\end{array} \right.
$$

Zauważ, że $\mathbf{r}$ może zostać obliczony jako dominujący wektor własny macierzy $\mathbf{A}$ za pomocą metody potęgowej (dominujący wektor własny $\mathbf{q_1}$ znormalizowany za pomocą normy $L1$). Przetestuj poprawność obliczeń korzystając z $3$ dowolnych silnie spójnych grafów skierowanych o liczbie wierzchołków większej niż $10$.

Funkcja zwracająca rangi poszczególnych wierzchołków na podstawie macierzy sąsiedztwa:

In [9]:
def pagerank_simple(A):
    _, q = np.linalg.eig(A)
    
    return np.linalg.norm(q, ord=1, axis=1)

Funkcja realizująca zadanie dla grafu skierowanego silnie spójnego o $n$ wierzchołkach:

In [10]:
def run_pagerank_simple(n, d, steps):
    A = generate_graph(n)
    AT = transform_graph(A) * d
    
    ranks = pagerank_simple(AT)
    trace, occurences = graph_inspect(A, ranks, steps)
    
    for node, (rank, occurence) in enumerate(zip(ranks, occurences)):
        print(f'node {node}\t- with rank {rank} \t- visited {occurence} times')

Wynik działania algorytmu dla $n=15$ oraz $d=0.8$:

In [11]:
run_pagerank_simple(15, 0.8, 10000)

node 0	- with rank 1.1750320840572346 	- visited 353 times
node 1	- with rank 3.3806094336969843 	- visited 903 times
node 2	- with rank 2.193180159685662 	- visited 601 times
node 3	- with rank 3.6214772267485333 	- visited 966 times
node 4	- with rank 3.325111963136659 	- visited 885 times
node 5	- with rank 2.0457019319911707 	- visited 542 times
node 6	- with rank 2.556839064296388 	- visited 679 times
node 7	- with rank 2.923646371927955 	- visited 810 times
node 8	- with rank 2.1093538324656573 	- visited 597 times
node 9	- with rank 3.0118818822369073 	- visited 806 times
node 10	- with rank 2.4665423341131367 	- visited 676 times
node 11	- with rank 1.8598505779767287 	- visited 477 times
node 12	- with rank 1.7378862317639285 	- visited 458 times
node 13	- with rank 2.311233590723353 	- visited 609 times
node 14	- with rank 2.365977696801183 	- visited 639 times


Wynik działania algorytmu dla $n=15$ oraz $d=0.6$:

In [12]:
run_pagerank_simple(15, 0.6, 10000)

node 0	- with rank 1.5362757700593097 	- visited 380 times
node 1	- with rank 2.243866145963325 	- visited 565 times
node 2	- with rank 2.1216717303352555 	- visited 492 times
node 3	- with rank 4.184829097227355 	- visited 943 times
node 4	- with rank 4.139588484753634 	- visited 947 times
node 5	- with rank 3.2435372555233486 	- visited 725 times
node 6	- with rank 3.428712250122417 	- visited 782 times
node 7	- with rank 2.0674835458048197 	- visited 519 times
node 8	- with rank 2.98929017936799 	- visited 703 times
node 9	- with rank 2.80831694473335 	- visited 655 times
node 10	- with rank 2.741615030512363 	- visited 639 times
node 11	- with rank 3.1557740678874024 	- visited 711 times
node 12	- with rank 3.427569267031881 	- visited 801 times
node 13	- with rank 2.7352057792279645 	- visited 608 times
node 14	- with rank 2.297182216861308 	- visited 531 times


Wynik działania algorytmu dla $n=20$ oraz $d=0.6$:

In [13]:
run_pagerank_simple(20, 0.6, 10000)

node 0	- with rank 1.3098889756733305 	- visited 218 times
node 1	- with rank 2.249749385667339 	- visited 399 times
node 2	- with rank 3.7113650585613844 	- visited 619 times
node 3	- with rank 3.836910221038004 	- visited 638 times
node 4	- with rank 4.087516452615946 	- visited 683 times
node 5	- with rank 3.739118055359914 	- visited 650 times
node 6	- with rank 2.1057363349989533 	- visited 360 times
node 7	- with rank 2.5883456032893046 	- visited 471 times
node 8	- with rank 2.6477782948901463 	- visited 430 times
node 9	- with rank 2.2920373349258196 	- visited 373 times
node 10	- with rank 3.0068087358886424 	- visited 553 times
node 11	- with rank 2.4496067859612225 	- visited 388 times
node 12	- with rank 3.1012629563724454 	- visited 533 times
node 13	- with rank 2.4122334089061104 	- visited 418 times
node 14	- with rank 2.9536406612877233 	- visited 478 times
node 15	- with rank 2.4723786983568448 	- visited 451 times
node 16	- with rank 2.1000675221159315 	- visited 336 

Wynik działania algorytmu dla $n=25$ oraz $d=0.6$:

In [14]:
run_pagerank_simple(25, 0.6, 20000)

node 0	- with rank 1.5973117270291588 	- visited 368 times
node 1	- with rank 1.175713246186684 	- visited 307 times
node 2	- with rank 2.9391306008665525 	- visited 688 times
node 3	- with rank 4.471944750750196 	- visited 1059 times
node 4	- with rank 3.4450917701602815 	- visited 831 times
node 5	- with rank 4.360735860760785 	- visited 983 times
node 6	- with rank 4.273569482627378 	- visited 979 times
node 7	- with rank 4.068647947940198 	- visited 1008 times
node 8	- with rank 3.5391463886356735 	- visited 827 times
node 9	- with rank 3.1979371817823066 	- visited 765 times
node 10	- with rank 3.946562862839041 	- visited 895 times
node 11	- with rank 2.1928659788853926 	- visited 499 times
node 12	- with rank 2.987640389462582 	- visited 696 times
node 13	- with rank 4.887081575364392 	- visited 1114 times
node 14	- with rank 3.023098604343252 	- visited 697 times
node 15	- with rank 4.323871319209792 	- visited 1012 times
node 16	- with rank 5.119805723792004 	- visited 1212 ti

### Zadanie 2: Page Rank

Rozszerz model z poprzedniego zadania, dodając możliwość skoku do losowego wierzchołka grafu:

$$\mathbf{r}(u)=d\sum_{v\in{B_u}}{\frac{\mathbf{r}(v)}{N_v}}+(1-d)\mathbf{e}(u)$$

W zapisie macierzowym:

$$\mathbf{r}=(d\mathbf{A}+1(1-d)\mathbf{e}\otimes\mathbf{1})\mathbf{r}$$

1. $\mathbf{r}_0$
2. $\mathtt{do}$
3. $\mathbf{r}_{i+1}=\mathbf{Br}_i$
4. $d=\lVert\mathbf{r}_i\rVert_1-\lVert\mathbf{r}_{i+1}\rVert_1$
5. $\mathbf{r}_{i+1}=\mathbf{r}_{i+1}+d\mathbf{e}$
6. $\delta=\lVert\mathbf{r}_{i+1}-\mathbf{r}_i\rVert_1$
7. $\mathtt{while}$ $\delta>\epsilon$

Przetestuj działanie zaimplementowanego algorytmu Page Rank dla wybranych grafów
z bazy SNAP. Przetestuj różne wartości parametru $d$ $(0.9, 0.85, 0.75, 0.6, 0.5)$ oraz różne
postacie wektora $\mathbf{e}$, przykładowo $\mathbf{e}=\frac{1}{n}[1,1,...,1]$.

Funkcja zwracająca rangi wierzchołków na podstawie zadanych parametrów. Obliczenia są wykonywane na podstawie wzoru:

$$\mathbf{r}=(d\mathbf{A}+1(1-d)\mathbf{e}\otimes\mathbf{1})\mathbf{r}$$

In [15]:
def pagerank_ext(A, d, e, iterations=30):
    n = len(A)
    dAe = d * A + np.kron(np.linalg.norm((1 - d) * e, 1), np.ones(n))
    
    r = np.random.rand(n, 1)
    for i in range(iterations):
        r = dAe @ r
    
    return r.flatten()

Wszystkie testy zostaną wykonane na grafie z bazy SNAP, który jest umieszczony w pliku $\mathtt{p2p-gnutella.txt}$:

Ze względu na fakt, iż testowany graf jest grafem rzadkim i przechodzenie do kolejnych wierzchołków z uwzględnieniem ewentualnych powrotów w momencie ślepego zaułka jest mało efektywne - wynikami testów będą rangi pierwszych pięciu wierzchołków

In [16]:
A = load_graph('p2p-gnutella.txt')
n = len(A)

Test dla wektora $\mathbf{e}=\frac{1}{n}[1,1,...,1]$ oraz parametru $d=0.9$:

In [17]:
e = np.ones(n) / n
d = 0.9

for node, rank in enumerate(pagerank_ext(A, d, e, iterations=20)[0:5]):
    print(f'node {node}\t- with rank {rank}')

node 0	- with rank 5.353129648120256e+55
node 1	- with rank 5.277468965679694e+55
node 2	- with rank 5.277468965679694e+55
node 3	- with rank 5.352828349732102e+55
node 4	- with rank 5.352689038013629e+55


Test dla wektora $\mathbf{e}=\frac{1}{n}[1,1,...,1]$ oraz parametru $d=0.8$:

In [18]:
e = np.ones(n) / n
d = 0.85

for node, rank in enumerate(pagerank_ext(A, d, e, iterations=20)[0:5]):
    print(f'node {node}\t- with rank {rank}')

node 0	- with rank 1.7415042888031955e+59
node 1	- with rank 1.7259460387669562e+59
node 2	- with rank 1.7259460387669562e+59
node 3	- with rank 1.7414652172094685e+59
node 4	- with rank 1.7414471238276405e+59


Test dla wektora $\mathbf{e}=\frac{1}{n}[1,1,...,1]$ oraz parametru $d=0.75$:

In [19]:
e = np.ones(n) / n
d = 0.75

for node, rank in enumerate(pagerank_ext(A, d, e, iterations=20)[0:5]):
    print(f'node {node}\t- with rank {rank}')

node 0	- with rank 4.572706187189471e+63
node 1	- with rank 4.5510112942292484e+63
node 2	- with rank 4.5510112942292484e+63
node 3	- with rank 4.572677306735847e+63
node 4	- with rank 4.572663916316188e+63


Test dla wektora $\mathbf{e}=\frac{1}{n}[1,1,...,1]$ oraz parametru $d=0.6$:

In [20]:
e = np.ones(n) / n
d = 0.6

for node, rank in enumerate(pagerank_ext(A, d, e, iterations=20)[0:5]):
    print(f'node {node}\t- with rank {rank}')

node 0	- with rank 5.447794504384264e+67
node 1	- with rank 5.434848426704947e+67
node 2	- with rank 5.434848426704947e+67
node 3	- with rank 5.447785881230345e+67
node 4	- with rank 5.447781880361939e+67


Test dla wektora $\mathbf{e}=\frac{1}{n}[1,1,...,1]$ oraz parametru $d=0.5$:

In [21]:
e = np.ones(n) / n
d = 0.5

for node, rank in enumerate(pagerank_ext(A, d, e, iterations=20)[0:5]):
    print(f'node {node}\t- with rank {rank}')

node 0	- with rank 4.749459801357504e+69
node 1	- with rank 4.741931012836553e+69
node 2	- with rank 4.741931012836553e+69
node 3	- with rank 4.749456457362048e+69
node 4	- with rank 4.7494549055005586e+69


Test dla wektora $\mathbf{e}=\frac{1}{n^2}[1,1,...,1]$ oraz parametru $d=0.8$:

In [22]:
e = np.ones(n) / (n * n)
d = 0.8

for node, rank in enumerate(pagerank_ext(A, d, e, iterations=20)[0:5]):
    print(f'node {node}\t- with rank {rank}')

node 0	- with rank 4421336089788.077
node 1	- with rank 40710029847.785065
node 2	- with rank 40710029847.785065
node 3	- with rank 1401662791609.9731
node 4	- with rank 1440487757145.58


Test dla wektora $\mathbf{e}=\frac{1}{n^3}[1,1,...,1]$ oraz parametru $d=0.8$:

In [23]:
e = np.ones(n) / (n * n * n)
d = 0.8

for node, rank in enumerate(pagerank_ext(A, d, e, iterations=20)[0:5]):
    print(f'node {node}\t- with rank {rank}')

node 0	- with rank 2739291773585.8223
node 1	- with rank 3834622.1273957463
node 2	- with rank 3834622.1273957463
node 3	- with rank 786119818916.7188
node 4	- with rank 838430657583.3915


Test dla wektora $\mathbf{e}=\sqrt{n}[1,1,...,1]$ oraz parametru $d=0.8$:

In [24]:
e = np.ones(n) * math.sqrt(n)
d = 0.8

for node, rank in enumerate(pagerank_ext(A, d, e, iterations=20)[0:5]):
    print(f'node {node}\t- with rank {rank}')

node 0	- with rank 4.8146043273677044e+175
node 1	- with rank 4.814604266259871e+175
node 2	- with rank 4.814604266259871e+175
node 3	- with rank 4.8146043273677044e+175
node 4	- with rank 4.8146043273677044e+175


Test dla wektora $\mathbf{e}=\frac{\sqrt{n}}{n}[1,1,...,1]$ oraz parametru $d=0.8$:

In [25]:
e = np.ones(n) * math.sqrt(n) / n
d = 0.8

for node, rank in enumerate(pagerank_ext(A, d, e, iterations=20)[0:5]):
    print(f'node {node}\t- with rank {rank}')

node 0	- with rank 5.051461041432011e+99
node 1	- with rank 5.051057082618742e+99
node 2	- with rank 5.051057082618742e+99
node 3	- with rank 5.051461032386556e+99
node 4	- with rank 5.051461028186977e+99


Jak można zauważyć wartości rang wierzchołków zmieniają się zarówno poprzez zmianę parametru $d$ jak i zmianę wektora $\mathbf{e}$.